In [ ]:
from ultralytics import YOLO
import cv2
import os
import matplotlib.pyplot as plt
import random

# Load the pretrained YOLO model
model = YOLO("yolov9e.pt")

# Specify the path to the 'images' folder
folder_path = "./images"

# Specify the output folder for processed images
output_folder = "./Processed images"

# Check if the output folder exists, and if not, create it
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# List all image files in the folder
image_files = [f for f in os.listdir(folder_path) if f.endswith('.png')]

def process_image(image_file):
    image_path = os.path.join(folder_path, image_file)
    img = cv2.imread(image_path)
    results = model(img)
    return img, results

def draw_bounding_boxes(image, results):
    # Assume class index for 'car' is 2; this should be confirmed
    car_class_index = 2

    for *xyxy, conf, cls in results.xyxy[0]:
        if int(cls) == car_class_index:
            x1, y1, x2, y2 = map(int, xyxy)
            image = cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            centroid_x = (x1 + x2) // 2
            centroid_y = (y1 + y2) // 2
            image = cv2.circle(image, (centroid_x, centroid_y), 8, (255, 0, 0), -1)
    
    return image

# Process and save all images
for image_file in image_files:
    img, results = process_image(image_file)
    processed_image = draw_bounding_boxes(img, results)
    output_path = os.path.join(output_folder, "processed_" + image_file)
    cv2.imwrite(output_path, processed_image)

# Optionally display some example images
def display_example_images(output_folder, num_images=3):
    processed_image_paths = [os.path.join(output_folder, f) for f in os.listdir(output_folder) if f.startswith("processed_")]
    
    if num_images > len(processed_image_paths):
        num_images = len(processed_image_paths)

    random.shuffle(processed_image_paths)

    for i in range(num_images):
        image = cv2.imread(processed_image_paths[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(8, 8))
        plt.imshow(image)
        plt.axis('off')
        plt.title(f"Processed Image {i+1}")
        plt.show()
